In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.getcwd()
os.chdir('/content/drive/My Drive/Colab Notebooks')
os.getcwd()

'/content/drive/My Drive/Colab Notebooks'

# Install the necessary packages to run this script

In [4]:
!pip install transformers datasets torch scikit-learn numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### Load the necessary packages to run the script

In [5]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, RobertaConfig
from datasets import load_dataset
import torch
from sklearn.metrics import accuracy_score
from modifiers.ModifiedRobertaWithAdaptersV5 import ModifiedRobertaForSequenceClassification
from transformers import TrainerCallback


Now lets check your GPU availability and load some sanity checkers. By default you should be using your gpu for this assignment if you have one available.

In [6]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


## **1.2: Load Data**
Loading the ag_news dataset

In [7]:
# Load AG_NEWS dataset
dataset = load_dataset("fancyzhx/ag_news")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [8]:
# load the tokenizer for Roberta
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [9]:
# define the function for tokenizing the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding = "max_length", truncation= True, max_length= 512)

# Apply the tokenizer to the datasets
tokenized_dataset = dataset.map(tokenize_function, batched= True)

# Set the format of the dataset to return PyTorch tensors
tokenized_datasets = tokenized_dataset.remove_columns(["text"])
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Split the dataset into train and test sets
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [10]:
small_train_dataset = train_dataset.select(range(1000))
small_test_dataset = test_dataset.select(range(100))

Setting the base model and the metric function to be used to evaluate the model

In [11]:
# # Load the pre-trained RoBERTa model
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4)
# def compute_metrics(p):
#     predictions, labels = p
#     predictions = torch.argmax(torch.tensor(predictions), dim=-1)
#     accuracy = accuracy_score(labels, predictions)
#     return {'accuracy': accuracy}

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # Get the predicted class by finding the index of the max logit

    # Accuracy
    acc = accuracy_score(labels, preds)

    # Precision, Recall, F1 Score (using macro, micro, or weighted average)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, zero_division=0.0, average="weighted")

    # # Confusion matrix (for multi-class, it returns a matrix)
    # cm = confusion_matrix(labels, preds)

    # Extract True Positives, False Positives, etc. for each class
    # Here, we will return a dictionary with confusion matrix components per class (if needed)
    # tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (None, None, None, None)

    # For multi-class, it's more useful to look at the entire confusion matrix
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        # 'confusion_matrix': cm.tolist(),  # Return the confusion matrix for more insight
    }



model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Setting the training options and hyper-parameter settings. This is going to be the same across all experiments

In [12]:
training_args = TrainingArguments(
    output_dir='./results',
    run_name="roberta_ag_news_ablation_adapter",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.001,
    lr_scheduler_type="linear",

    logging_dir='./logs',
    logging_steps=100,
    save_strategy="epoch",               # Save checkpoints every epoch
    load_best_model_at_end=True,         # Load best model after training
    metric_for_best_model="accuracy",    # Metric to monitor
    fp16=True,
    # report_to=["none"],  # Disable W&B logging
    #logging_dir='./logs',
    #logging_steps=10,
    # report_to=["none"],  # Disable W&B logging
    # fp16=True  # Enables mixed precision
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Please run this below cell when we want to train on all the data. Current setup is just to test if the training is working





In [13]:
trainAllData = True
if trainAllData:
  small_train_dataset = train_dataset
  small_test_dataset = test_dataset

The Following section setup the different types of Models for training on the ag_news dataset
# Base model with the classification head is finetuned.
The whole model is finetuned to ag_news dataset

In [14]:
# This will display the model structure and the layer structure
base_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [15]:
training_logs = []

class CustomCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        logs = logs or {}
        if any(key.startswith("eval_") for key in logs):
            training_logs.append({
                "epoch": state.epoch,
                "loss": logs.get("eval_loss", None),  # Get loss if available, otherwise None
                "accuracy": logs.get("eval_accuracy", None)
            })

trainer_base = Trainer(
    model=base_model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

trainer_base.add_callback(CustomCallback)

In [ ]:
#trainer_base.train()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# base_model.save_pretrained("/content/drive/My Drive/1_base_model_ag_news_ablation_with_adapter")



# Base model only the classification head is finetuned.
The classification head is finetuned to ag_news dataset

In [ ]:
# base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4)
# model = base_model
# for param in model.base_model.parameters():
#     param.requires_grad = False  # Freeze the encoder layers

In [ ]:
# trainer_base_ch_only = Trainer(
#     model=model,                         # the model to be trained
#     args=training_args,                  # training arguments
#     train_dataset=small_train_dataset,  # training dataset
#     eval_dataset=small_test_dataset,   # evaluation dataset
#     compute_metrics=compute_metrics,     # function for computing metrics
# )
# trainer_base_ch_only.add_callback(CustomCallback)

In [ ]:
# trainer_base_ch_only.train()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# model.save_pretrained("/content/drive/My Drive/2_base_model_ag_news_ablation_with_adapter")



# Custom model with no adapters used. All parameters fine tuned




In [ ]:
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4)
custom_model = ModifiedRobertaForSequenceClassification(base_model ,num_labels=4, adapter_hidden_dim=64, freeze_params = True, adapter_layers = [False] * 12)
# View the updated custom model. All encoder blocks should be similar and should have a pass through block instead of the adapter block
custom_model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModifiedRobertaForSequenceClassification(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (modified_layers): ModuleList(
    (0-11): 12 x ModifiedRobertaLayer(
      (adapter1): PassThroughBlock()
      (adapter2): PassThroughBlock()
      (attention): RobertaAttention(
        (self): RobertaSdpaSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): RobertaSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): La

In [ ]:
trainer_custom_model = Trainer(
    model=custom_model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

trainer_custom_model.add_callback(CustomCallback)

In [ ]:
trainer_custom_model.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.390000,1.386475,0.250000,0.062500,0.250000,0.100000
2,1.388400,1.386232,0.250000,0.062500,0.250000,0.100000
3,1.388100,1.386475,0.250000,0.062500,0.250000,0.100000
4,1.386700,1.386475,0.250000,0.062500,0.250000,0.100000
5,1.387800,1.386028,0.250000,0.062500,0.250000,0.100000


TrainOutput(global_step=37500, training_loss=1.3881151025390626, metrics={'train_runtime': 2871.7602, 'train_samples_per_second': 208.931, 'train_steps_per_second': 13.058, 'total_flos': 0.0, 'train_loss': 1.3881151025390626, 'epoch': 5.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# custom_model.save_pretrained("/content/drive/My Drive/3_base_custom_model_ag_news_ablation_with_adapter")
torch.save(custom_model.state_dict(), "/content/drive/My Drive/3_base_custom_model_ag_news_ablation_with_adapter.pt")





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Custom model with no adapters used. Classification head fine tuned

In [ ]:
# set-requires_grad will freeze all original layer parameters and only train the adapters and classification heads
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4)
custom_model = ModifiedRobertaForSequenceClassification(base_model ,num_labels=4, adapter_hidden_dim=64, freeze_params = True, adapter_layers = [False] * 12)
custom_model.set_requires_grad(False)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer_custom_model_ch_only = Trainer(
    model=custom_model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

trainer_custom_model_ch_only.add_callback(CustomCallback)

In [ ]:
trainer_custom_model_ch_only.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.939800,1.127238,0.535000,0.633336,0.535000,0.519204
2,0.920500,1.182985,0.546184,0.657637,0.546184,0.529714
3,0.904600,1.210906,0.545921,0.652638,0.545921,0.528291
4,0.891400,1.192123,0.554079,0.656932,0.554079,0.538918
5,0.878300,1.208453,0.552763,0.657740,0.552763,0.536695


TrainOutput(global_step=37500, training_loss=0.9398406026204427, metrics={'train_runtime': 1022.0286, 'train_samples_per_second': 587.068, 'train_steps_per_second': 36.692, 'total_flos': 0.0, 'train_loss': 0.9398406026204427, 'epoch': 5.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# custom_model.save_pretrained("/content/drive/My Drive/4_custom_model_ag_news_ablation_with_adapter")


torch.save(custom_model.state_dict(), "/content/drive/My Drive/4_custom_model_ag_news_ablation_with_adapter.pt")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Custom model with all layers with adapters used. Adapters and Classification head fine tuned only

In [16]:
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4)
model_all_adapters = ModifiedRobertaForSequenceClassification(base_model ,num_labels=4, adapter_hidden_dim=64, freeze_params = True, adapter_layers = [True] * 12)
# View the updated custom model. All encoder blocks should be similar and should have a pass through block instead of the adapter block
model_all_adapters.set_requires_grad(False)
model_all_adapters

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModifiedRobertaForSequenceClassification(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (modified_layers): ModuleList(
    (0-11): 12 x ModifiedRobertaLayer(
      (adapter1): AdapterBlock(
        (fc1): Linear(in_features=768, out_features=64, bias=True)
        (relu): ReLU()
        (fc2): Linear(in_features=64, out_features=768, bias=True)
      )
      (adapter2): AdapterBlock(
        (fc1): Linear(in_features=768, out_features=64, bias=True)
        (relu): ReLU()
        (fc2): Linear(in_features=64, out_features=768, bias=True)
      )
      (attention): RobertaAttention(
        (self): RobertaSdpaSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): 

In [17]:
trainer_model_all_adapters = Trainer(
    model=model_all_adapters,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

trainer_model_all_adapters.add_callback(CustomCallback)

In [18]:
trainer_model_all_adapters.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.387500,1.386963,0.250000,0.062500,0.250000,0.100000
2,1.387800,1.386719,0.250000,0.062500,0.250000,0.100000
3,1.386300,1.386475,0.250000,0.062500,0.250000,0.100000
4,1.387100,1.386230,0.250000,0.062500,0.250000,0.100000
5,1.386300,1.386230,0.250000,0.062500,0.250000,0.100000


TrainOutput(global_step=37500, training_loss=1.3868245572916666, metrics={'train_runtime': 2354.9637, 'train_samples_per_second': 254.781, 'train_steps_per_second': 15.924, 'total_flos': 0.0, 'train_loss': 1.3868245572916666, 'epoch': 5.0})

In [20]:
from google.colab import drive
drive.mount('/content/drive')

torch.save(model_all_adapters.state_dict(), "/content/drive/My Drive/5_model_all_adapters_ag_news_ablation_with_adapter.pt")

# model_all_adapters.save_pretrained("/content/drive/My Drive/5_model_all_adapters_ag_news_ablation_with_adapter")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Custom model with lower layers(2) with adapters used. Adapters and Classification head fine tuned only




In [21]:
adpt_lyrs = [False]*12
adpt_lyrs[0:2] = [True]*2
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4)
model_lower_adapters = ModifiedRobertaForSequenceClassification(base_model ,num_labels=4, adapter_hidden_dim=64, freeze_params = True, adapter_layers = adpt_lyrs)
model_lower_adapters.set_requires_grad(False)
# View the updated custom model. All encoder blocks should be similar and should have a pass through block instead of the adapter block
model_lower_adapters

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModifiedRobertaForSequenceClassification(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (modified_layers): ModuleList(
    (0-1): 2 x ModifiedRobertaLayer(
      (adapter1): AdapterBlock(
        (fc1): Linear(in_features=768, out_features=64, bias=True)
        (relu): ReLU()
        (fc2): Linear(in_features=64, out_features=768, bias=True)
      )
      (adapter2): AdapterBlock(
        (fc1): Linear(in_features=768, out_features=64, bias=True)
        (relu): ReLU()
        (fc2): Linear(in_features=64, out_features=768, bias=True)
      )
      (attention): RobertaAttention(
        (self): RobertaSdpaSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Li

In [22]:
trainer_model_lower_adapters = Trainer(
    model=model_lower_adapters,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

trainer_model_lower_adapters.add_callback(CustomCallback)

In [23]:
trainer_model_lower_adapters.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.395400,1.388083,0.250000,0.062500,0.250000,0.100000
2,0.557300,0.574046,0.803289,0.808549,0.803289,0.801463
3,0.342400,0.409241,0.880921,0.883195,0.880921,0.881270
4,0.287900,0.333564,0.896974,0.897984,0.896974,0.897068
5,0.279100,0.315887,0.902105,0.902517,0.902105,0.902047


TrainOutput(global_step=37500, training_loss=0.7214542170715332, metrics={'train_runtime': 2039.2798, 'train_samples_per_second': 294.222, 'train_steps_per_second': 18.389, 'total_flos': 0.0, 'train_loss': 0.7214542170715332, 'epoch': 5.0})

In [24]:
from google.colab import drive
drive.mount('/content/drive')

torch.save(model_lower_adapters.state_dict(), "/content/drive/My Drive/6_model_lower_adapters_ag_news_ablation_with_adapter.pt")

# model_lower_adapters.save_pretrained("/content/drive/My Drive/6_model_lower_adapters_ag_news_ablation_with_adapter")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Custom model with higher layers(2) with adapters used. Adapters and Classification head fine tuned only

In [25]:
adpt_lyrs = [False]*12
adpt_lyrs[10:] = [True]*2
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4)
model_higher_adapters = ModifiedRobertaForSequenceClassification(base_model ,num_labels=4, adapter_hidden_dim=64, freeze_params = True, adapter_layers = adpt_lyrs)
model_higher_adapters.set_requires_grad(False)
# View the updated custom model. All encoder blocks should be similar and should have a pass through block instead of the adapter block
model_higher_adapters

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModifiedRobertaForSequenceClassification(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (modified_layers): ModuleList(
    (0-9): 10 x ModifiedRobertaLayer(
      (adapter1): PassThroughBlock()
      (adapter2): PassThroughBlock()
      (attention): RobertaAttention(
        (self): RobertaSdpaSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): RobertaSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): Lay

In [26]:
trainer_model_higher_adapters = Trainer(
    model=model_higher_adapters,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

trainer_model_higher_adapters.add_callback(CustomCallback)

In [27]:
trainer_model_higher_adapters.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.446100,0.398992,0.871447,0.871883,0.871447,0.871490
2,0.398200,0.367000,0.880000,0.880048,0.880000,0.880016
3,0.358500,0.350859,0.887368,0.887701,0.887368,0.887358
4,0.314600,0.331120,0.888421,0.892563,0.888421,0.888371
5,0.273700,0.324467,0.892500,0.894852,0.892500,0.892406


TrainOutput(global_step=37500, training_loss=0.42560977376302084, metrics={'train_runtime': 1216.7465, 'train_samples_per_second': 493.118, 'train_steps_per_second': 30.82, 'total_flos': 0.0, 'train_loss': 0.42560977376302084, 'epoch': 5.0})

In [28]:
from google.colab import drive
drive.mount('/content/drive')

torch.save(model_higher_adapters.state_dict(), "/content/drive/My Drive/7_model_higher_adapters_ag_news_ablation_with_adapter.pt")

# model_higher_adapters.save_pretrained("/content/drive/My Drive/7_model_higher_adapters_ag_news_ablation_with_adapter")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
